### All needed imports

In [17]:
import os
import cv2
import numpy as np
import tensorflow as tf
assert tf.__version__.startswith('2')
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense


### Limit GPU Usage

In [18]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


### Preprocessing Data for Model 1

#### Video Paths

In [19]:
falling_paths = [
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall2.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall3.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall4.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall1.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall5.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall6.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall7.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall8.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall9.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall10.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall11.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall12.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall13.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall14.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall15.mp4"
                 ]

default_paths = [
                 "./../datasets/vids/splitted/new_still/resized_logitech-default1.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default2.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default3.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default4.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default5.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default6.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default7.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default8.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default9.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default10.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default11.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default12.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default13.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default14.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default15.mp4"
                 ]

test_falling_paths = [
                "./../datasets/vids/testdata/moving/resized-test-fall1.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall2.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall3.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall4.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall5.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall6.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall7.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall8.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall9.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall10.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall11.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall12.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall13.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall14.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall15.mp4"
]

test_default_paths = [
                "./../datasets/vids/testdata/still/resized-test-default1.mp4",
                "./../datasets/vids/testdata/still/resized-test-default2.mp4",
                "./../datasets/vids/testdata/still/resized-test-default3.mp4",
                "./../datasets/vids/testdata/still/resized-test-default4.mp4",
                "./../datasets/vids/testdata/still/resized-test-default5.mp4",
                "./../datasets/vids/testdata/still/resized-test-default6.mp4",
                "./../datasets/vids/testdata/still/resized-test-default7.mp4",
                "./../datasets/vids/testdata/still/resized-test-default8.mp4",
                "./../datasets/vids/testdata/still/resized-test-default9.mp4",
                "./../datasets/vids/testdata/still/resized-test-default10.mp4",
                "./../datasets/vids/testdata/still/resized-test-default11.mp4",
                "./../datasets/vids/testdata/still/resized-test-default12.mp4",
                "./../datasets/vids/testdata/still/resized-test-default13.mp4",
                "./../datasets/vids/testdata/still/resized-test-default14.mp4",
                "./../datasets/vids/testdata/still/resized-test-default15.mp4"
]

#### Processing code, returns frame (num_frames, 224,224,3) and frame_diffs (num_frames, 224,224)

In [20]:
# Code to apply the same random transformation 

In [21]:
def process_videos(video_paths, label):
    frames = []
    frame_diffs = []
    labels = []
    
    for path in video_paths:
        video_cap = cv2.VideoCapture(path)
        
        prev_gray_frame = None
        
        while video_cap.isOpened():
            ret, frame = video_cap.read()
            
            if not ret:
                break
            
            # Resize and convert frame to RGB
            frame_resized = cv2.resize(frame, (224, 224))
            frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
            
            # Convert frame to grayscale
            gray_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

            if prev_gray_frame is None:
                frame_diff = np.zeros_like(gray_frame, dtype=np.float32)
            else:
                frame_diff = cv2.absdiff(prev_gray_frame, gray_frame)
                # frame_diff = cv2.absdiff(prev_gray_frame, gray_frame) / 255.0
            
            prev_gray_frame = gray_frame
            
            frames.append(frame_rgb)
            frame_diffs.append(frame_diff)
            labels.append(label)  # 0 for still, 1 for moving
            
            
        video_cap.release()

    return np.array(frames), np.array(frame_diffs), np.array(labels)

In [22]:
# Reshaping to 224 x 224 x 4 for the convolutional model
def combine_frames_and_diffs(frames, frame_diffs):
    # frame_diffs dimension
    # print(frame_diffs.shape)
    frame_diffs_expanded = np.expand_dims(frame_diffs, axis=-1) # Add an extra dimension
    # frame_diffs_expanded dimension
    # print(frame_diffs_expanded.shape)
    combined_input = np.concatenate([frames, frame_diffs_expanded], axis=-1) # Concatenate along the last axis
    # combined_input dimension
    # print(combined_input.shape)
    
    # Visualize the difference
    # cv2.imwrite('img_still.png', combined_input[10])
    return combined_input

In [23]:
# Generate the frames and frame_diffs array for still
still_frames, still_diff, still_labels = process_videos(default_paths, 0)
test_still_frames, test_still_diff, test_still_labels = process_videos(test_default_paths, 0)
# print(test_still_frames.shape)
falling_frames, falling_diff, falling_labels = process_videos(falling_paths, 1)
test_falling_frames, test_falling_diff, test_falling_labels = process_videos(test_falling_paths, 1)
# print(test_falling_frames.shape)

# Combine them
concatenate_frames = np.concatenate([still_frames, falling_frames], axis = 0)
# print(concatenate_frames.shape)
concatenate_diff =  np.concatenate([still_diff, falling_diff], axis = 0)
# print(concatenate_diff.shape)
concatenate_labels =  np.concatenate([still_labels, falling_labels], axis = 0)
# print(concatenate_labels.shape)

# Create 224x224x4 shape for the model
combined_input = combine_frames_and_diffs(concatenate_frames, concatenate_diff)
# print(combined_input)

# Combine them
test_concatenate_frames = np.concatenate([test_still_frames, test_falling_frames], axis = 0)
test_concatenate_diff =  np.concatenate([test_still_diff, test_falling_diff], axis = 0)
test_concatenate_labels =  np.concatenate([test_still_labels, test_falling_labels], axis = 0)
print(test_concatenate_frames.shape)
# Create 224x224x4 shape for the model
test_combined_input = combine_frames_and_diffs(test_concatenate_frames, test_concatenate_diff)
print(test_combined_input.shape)

# Shuffle the data if needed
indices = np.arange(combined_input.shape[0])
np.random.shuffle(indices)
combined_input = combined_input[indices]
concatenate_labels = concatenate_labels[indices]

#data augmentation 
data_augmentation = tf.keras.Sequential([
    # layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomContrast(factor=0.2)

])


(1015, 224, 224, 3)
(1015, 224, 224, 4)


In [24]:
# Create a TensorFlow dataset
with tf.device('/cpu:0'):
    dataset = tf.data.Dataset.from_tensor_slices((combined_input, concatenate_labels))
    dataset = dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)
    test_dataset = tf.data.Dataset.from_tensor_slices((test_combined_input, test_concatenate_labels))
    test_dataset = test_dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)

# Split the dataset into training and validation
train_dataset = dataset.take(int(0.8 * len(dataset)))
val_dataset = dataset.skip(int(0.8 * len(dataset)))

# dataset = tf.data.Dataset.from_tensor_slices((combined_input, concatenate_labels))
# Batch and prefetch if needed
# dataset = dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)

#### Create the model

In [28]:
def create_model():
    combined_input = Input(shape=(224, 224, 4), name='combined_input')

    # Apply data augmentation here
    x = data_augmentation(combined_input)

    # Convolutional layers for image processing
    # Convolutional layers are useful for learning spatial hierarchies and detecting features 
    x = Conv2D(32, (3, 3), activation='relu')(x) #  Convolutional layers extract spatial features from the image
    x = MaxPooling2D((2, 2))(x) #  Max-pooling layers reduce the size of the feature maps.
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    
    # Fully connected layers: These layers learn to make decisions based on the features extracted by the convolutional layers
    x = Flatten()(x) # This line flattens the 2D feature maps into a 1D vector
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)  # Single output with sigmoid activation

    model = Model(inputs=combined_input, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Binary cross-entropy loss
    
    return model
    
model = create_model()
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 combined_input (InputLayer)  [(None, 224, 224, 4)]    0         
                                                                 
 sequential_2 (Sequential)   (None, 224, 224, 4)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 222, 222, 32)      1184      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                       

In [29]:
model.fit(train_dataset, validation_data=val_dataset, epochs=5)

Epoch 1/5
54/54 [==============================] - 3s 46ms/step - loss: 12.1579 - accuracy: 0.7164 - val_loss: 0.3959 - val_accuracy: 0.8750
Epoch 2/5
54/54 [==============================] - 2s 44ms/step - loss: 0.4161 - accuracy: 0.8727 - val_loss: 0.3376 - val_accuracy: 0.8843
Epoch 3/5
54/54 [==============================] - 2s 44ms/step - loss: 0.2254 - accuracy: 0.9271 - val_loss: 0.2489 - val_accuracy: 0.9167
Epoch 4/5
54/54 [==============================] - 2s 42ms/step - loss: 0.2009 - accuracy: 0.9282 - val_loss: 0.1825 - val_accuracy: 0.9352
Epoch 5/5
54/54 [==============================] - 2s 44ms/step - loss: 0.1437 - accuracy: 0.9549 - val_loss: 0.1172 - val_accuracy: 0.9630


#### Save the model

In [30]:
model.evaluate(test_dataset)
model.save("m1")

64/64 [==============================] - 2s 14ms/step - loss: 0.9142 - accuracy: 0.7202
INFO:tensorflow:Assets written to: m1/assets
